## Problem 3: How long distance individuals have travelled? (8 points)

In this problem the aim is to calculate the "distance" in meters that the individuals have travelled according the social media posts (Euclidean distances between points). In this problem, we will need the `userid` -column an the points created in the previous problem. You will need the shapefile `Kruger_posts.shp` generated in Problem 2 as input file.

Our goal is to answer these questions based on the input data:
- What was the shortest distance travelled in meters?
- What was the mean distance travelled in meters?
- What was the maximum distance travelled in meters?

**In your code, you should first:**
 - Import required modules.
 - Read in the shapefile as a geodataframe called `data`
 - Reproject the data from WGS84 projection into `EPSG:32735` -projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into metric system.
 
*Store the result in a variable called `data`*!

In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from pyproj import CRS
import matplotlib.pyplot as plt

# Read the file
fp = "Kruger_posts.shp"
data = gpd.read_file(fp)

data.crs


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

- Check the crs of the input data. If this information is missing, set it as epsg:4326 (WGS84).
- Reproject the data from WGS84 to `EPSG:32735` -projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into metric system. (don't create a new variable, update the existing variable `data`!)"

In [2]:
# Reproject the data
data = data.to_crs(epsg=32735) #funcion nativa de geoDtaframe
                # se puede usar crs en lufar de epsg

In [3]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the data
print(data.head())

         lat        lon         timestamp    userid  \
0 -24.980792  31.484633  2015-07-07 03:02  66487960   
1 -25.499225  31.508906  2015-07-07 03:18  65281761   
2 -24.342578  30.930866  2015-03-07 03:38  90916112   
3 -24.854614  31.519718  2015-10-07 05:04  37959089   
4 -24.921069  31.520836  2015-10-07 05:19  27793716   

                         geometry  
0  POINT (952912.890 7229683.258)  
1  POINT (953433.223 7172080.632)  
2  POINT (898955.144 7302197.408)  
3  POINT (956927.218 7243564.942)  
4  POINT (956794.955 7236187.926)  


In [4]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check that the crs is correct after re-projecting (should be epsg:32735)
print(data.crs)

epsg:32735


 - Group the data by userid

In [5]:
# Group the data by class
grouped = data.groupby('userid')

In [11]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the number of groups:
assert len(grouped.groups) == data["userid"].nunique(), "Number of groups should match number of unique users!"

**Create LineString objects for each user connecting the points from oldest to latest:**

*Suggested steps:*
- Create an empty DataFrame called `movements`. 
- Create an empty column "geometry"
- Use a for-loop where you iterate over the grouped object. For each user's data: 
    - [sort](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html) the rows by timestamp 
    - create a LineString object based on the user's points
    - Add the LineString to the geometry column of the `movements` dataframe. You can also add the `userid` in a separate column (or use the userid as index).
- Convert `movements` into a `GeoDataFrame` (you can replace the DataFrame created in the previous steps with the GeoDataFrame). Remember to set the `geometry` column.
- Set the CRS of the ``movements`` GeoDataFrame as ``EPSG:32735`` 

In [38]:
from shapely.geometry import Point, Polygon, LineString

movements = pd.DataFrame()
movements['geometry'] = None
i = 0

for key,group in grouped:
    if len(group) > 1:
        #movements['geometry'].append(LineString(group['geometry'].tolist())) doesnt work!
        Point_list = []
        group.sort_values(by='timestamp', ascending=True)
        group.apply(lambda row: Point_list.append(row['geometry']),axis = 1)
        movements.at[i,'geometry'] = LineString(Point_list) 
        movements.at[i,'userid'] =key
        i += 1


movements = gpd.GeoDataFrame(movements, geometry='geometry', crs=CRS.from_epsg(32735).to_wkt())
        
#movements = gpd.GeoDataFrame(movements, geometry='geometry', crs=CRS.from_epsg(32735).to_wkt()) 
                
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
#raise NotImplementedError()


In [40]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the result
print(type(movements))
print(movements.crs)
print(movements["geometry"].head())

<class 'geopandas.geodataframe.GeoDataFrame'>
PROJCRS["WGS 84 / UTM zone 35S",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["UTM zone 35S",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",27,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Scale factor at natural origin",0.9996,SCALEUNIT["unity",1],ID["EPSG",8805]],PARAMETER["False easting",500000,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",10000000,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["(E)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["(N)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - S hemisphere - 24°E to 30°E - by country"],BBOX[-80,24,0,30]],ID["EPS

In [ ]:
movements

**Finally:**
- Check once more the crs definition of your dataframe (should be epsg:32735, define the correct crs if this information is missing)
- Calculate the lenghts of the lines into a new column called ``distance`` in ``movements`` GeoDataFrame.

In [42]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
# raise NotImplementedError()
movements.crs
movements['distance'] = movements['geometry'].length

In [43]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the output
movements.head()

,geometry,userid,distance
0,"LINESTRING (939011.113 7254636.121, 942231.630...",16301.0,195251.395657
1,"LINESTRING (905394.500 7193375.148, 905394.500...",45136.0,0.000000
2,"LINESTRING (963788.403 7228015.063, 944551.607...",50136.0,254702.529630
3,"LINESTRING (902800.817 7192546.975, 902800.839...",88775.0,0.080245
4,"LINESTRING (959332.961 7219877.715, 963788.403...",88918.0,9277.252211


You should now be able to print answers to the following questions: 

 - What was the shortest distance travelled in meters?
 - What was the mean distance travelled in meters?
 - What was the maximum distance travelled in meters?

In [52]:
#movements[movements['distance'] == movements['distance'].mean()]
#movements[movements['distance'] == movements['distance'].min()]
#movements[movements['distance'] == movements['distance'].max()]

print('The average distance in users path is ',round(movements['distance'].mean(),1),' m')
print('The minimum distance in users path is ',round(movements['distance'].min(),1),' m')
print('The maximum distance in users path is ',round(movements['distance'].max(),1),' m')

The average distance in users path is  69090.4  m
The minimum distance in users path is  0.0  m
The maximum distance in users path is  4535319.0  m


- Finally, save the movements of into a Shapefile called ``some_movements.shp``

In [53]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
#raise NotImplementedError()
# Save the GeoDataFrame into a new Shapefile called 'polygon.shp'.
fp = 'some_movements.shp'
movements.to_file(fp)

In [54]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

import os

#Check if output file exists
assert os.path.isfile(fp), "Output file does not exits."

That's all for this week!